Bishop et al: https://www.nature.com/articles/nphys1154

Qutip reference:
https://nbviewer.jupyter.org/github/qutip/qutip-notebooks/blob/master/examples/rabi-oscillations.ipynb
http://qutip.org/docs/3.1.0/apidoc/functions.html

Koch 2007: https://arxiv.org/pdf/cond-mat/0703002.pdf

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize, stats, integrate, stats
from qutip import *
# Note to self: ^ IS NOT POWER, ** IS POWER

In [24]:
# Use hbar = 1

# number of cavity states
Nr = 5
# number of transmon states
Nj = 5
# cavity/resonator frequency
wr = 1.0  * 2 * np.pi
# transmon frequency (temporary)
wj = 2 * np.pi * np.asarray(range(Nj))
# transmon-resonator coupling strength
go = np.pi*347*10**6
gj = go * np.asarray([np.sqrt(j+1) for j in range(Nj)])
# drive frequency (temporary)
wd = 2 * np.pi * 1.0
# drive strength
xi = 0.001

# cavity dissipation rate
kappa = 300*10**3*2*np.pi
# transmon relaxation rate (2pi/T1)
gamma1 = 2*np.pi/(1.7*10**-9)
# transmon dephasing rate
gammaphi = 0.0
# temperature in frequency units
n_th_a = 0.0
# relative strengths of damping
alphaj = gj/g0
betaj = 2*epj/(epj[1]-epj[0])

use_rwa = True

# Transmon constants (temporary)
EC = 1
EJ = 10

In [23]:
def comm(a, b):
    return(a*b - b*a)

In [12]:
# Transmon states from perturbation theory (Koch p. 19)
b = destroy(Nj) # pure SHO operators
Hsho = np.sqrt(8*EC*EJ)*(b.dag()*b + 1/2)-EJ
evals, ekets = Hsho.eigenstates()
# print(evals)
ECorrect = [evals[j] - EC/12*(6*j**2 + 6*j + 3) for j in range(Nj)]
# print(ECorrected)
jketsCorrect = np.copy(ekets)
jkets1 = np.asarray([0*basis(Nj,0)]*Nj, dtype=object)
for j in range(Nj):
    for i in range(Nj):
        if i == j: continue
        quartic = (b+b.dag())**4
        element = quartic.matrix_element(ekets[i], ekets[j])
        jkets1[j] += (element*ekets[i])/(ECorrect[i]-ECorrect[j])
    jkets1[j] *= -EC/12
#     print(jkets1[j])
jketsCorrect += jkets1
# print("EKETS", ekets)
# print()
# print("JKETS", jketsCorrected)

jkets = jketsCorrect

In [17]:
# JC with coherent drive (Bishop p. 1)
a = tensor(destroy(Nr), qeye(Nj)) # cavity op
def Hjc(t):
    Hjc = wr*a.dag()*a
    for j in range(Nj):
        Hjc += wj[j] * tensor(qeye(Nr), ket2dm(jkets[j]))
    for j in range(Nj-1):
        temp = a*tensor(qeye(Nr), jkets[j+1]*jkets[j].dag())
        Hjc += gj[j]*(temp + temp.dag())
    temp = a.dag()*np.exp(-1j*wd*t)
    Hjc += xi*(temp + temp.dag())
    return(Hjc)
# Hjc(0)

In [ ]:
# Master eqn solver

# Damping terms
c_op_list = []
c_op_list.append(kappa*lindblad_dissipator(a))

temp 
for j in range(Nj-1):
    temp = a*tensor(qeye(Nr), jkets[j+1]*jkets[j].dag())
    Hjc += gj[j]*(temp + temp.dag())
c_op_list.append(gamma1*lindblad_dissipator(temp))